# Alternate Z02FH Analysis Approach

In [48]:
# This is an intersting alternate approach for reading in the battery data
import glob
import os
import pandas as pd
from pathlib import Path
from functools import partial

prs = partial(pd.read_csv, 
              sep="\t", 
              header=0, 
              skiprows=4, 
              usecols=['Rec#','Cyc#','Step','TestTime','StepTime','Amp-hr','Amps','Volts','State']
             )
path=os.path.join(os.getcwd(), "Z02_Test")  # This reminds me of what I have to do with PQ
files = Path(path).glob('*.txt')            # Making a path object provides all sorts of great methods

df = pd.concat((prs(f).assign(filename=f.stem) for f in files), ignore_index=True)

In [49]:
df

,Rec#,Cyc#,Step,TestTime,StepTime,Amp-hr,Amps,Volts,State,filename
0,1,0,1,0d 00:00:0,0d 00:00:0,0.00000,0.0000,3.79103,R,MB_U01_MZ02F_D420
1,2,0,1,0d 00:01:0,0d 00:01:0,0.00000,0.0000,3.79118,R,MB_U01_MZ02F_D420
2,3,1,4,0d 00:01:0,0d 00:00:0,0.00000,0.0005,3.78630,P,MB_U01_MZ02F_D420
3,4,1,5,0d 00:01:0.00999999977648258,0d 00:00:0.00999999977648258,0.00000,0.0000,3.79042,D,MB_U01_MZ02F_D420
4,5,1,5,0d 00:02:0.00999999977648258,0d 00:01:0.00999999977648258,0.00001,0.0005,3.78233,D,MB_U01_MZ02F_D420
...,...,...,...,...,...,...,...,...,...,...
39779,7139,5,5,4d 22:25:6.53000020980835,0d 12:18:0.0399999991059303,0.00615,0.0005,3.85199,D,MB_U03_MZ02F_D435
39780,7140,5,5,4d 22:26:6.53000020980835,0d 12:19:0.0399999991059303,0.00616,0.0005,3.85168,D,MB_U03_MZ02F_D435
39781,7141,5,5,4d 22:27:6.53000020980835,0d 12:20:0.0399999991059303,0.00617,0.0005,3.85130,D,MB_U03_MZ02F_D435
39782,7142,5,5,4d 22:28:6.53000020980835,0d 12:21:0.0399999991059303,0.00618,0.0005,3.85084,D,MB_U03_MZ02F_D435
